pyarrow, fastparquet 설치 필요

In [3]:
# 기상청 단기예보 조회서비스 (최근 3일 데이터만 가능)
# X,Y 좌표별 데이터 수집
# *일일 트래픽 10000건까지
# 코드로 돌릴때 인증키(디코딩 인증키): BL2wBSzTux87JT8DVgN59Hsiz/po3FS/yt4YE+HT6zoxiJFKMuN3oEZ0XKMOhSp64HiDZSO/gsI17IfFUPtX1g==
# 직접 URL에서 확인할때 인증키(인코딩 인증키): BL2wBSzTux87JT8DVgN59Hsiz%2Fpo3FS%2Fyt4YE%2BHT6zoxiJFKMuN3oEZ0XKMOhSp64HiDZSO%2FgsI17IfFUPtX1g%3D%3D


import requests
import pandas as pd
import datetime
import time
import json
import glob
import os
import sys
import glob

#---------------------------------------------------------------
# TEMP 폴더에 임시파일 저장
if not os.path.exists('temp'):
    os.mkdir('temp')

dict_args = dict()
dict_args['today'] = datetime.datetime.today().strftime('%y%m%d_%H%M')    # 오늘날짜
dict_args['tempPath'] = 'C:\\kim\\weather\\temp\\'     # 임시 파일은 이 경로에 저장
dict_args['resultPath'] = 'C:\\kim\\weather\\'      # 최종 결과와 코드 파일은 이 경로에 저장
dict_args['area_master'] = 'C:\\kim\\weather\\area_master_small.txt'     # 전국 X,Y 좌표
dict_args['url'] = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'   # 기상청 단기예보 조회서비스
dict_args['filename_to_save'] = ''
#---------------------------------------------------------------


# 기상청 API에서 데이터 수집
def get_data_api(p_args, **params): 
    
    today = p_args['today']
    nx = p_args['nx']
    ny = p_args['ny']
    filename = p_args['filename_to_save']
    url = p_args['url']
    tempPath = p_args['tempPath']
    
    response = requests.get(url, params=params)
    print('연결되었습니다. ', response)
    print(response.url)
    # print(response.content)

    jdata = json.loads(response.content)
    # print(json.dumps(jdata, indent=2)[:500])
    
    # 데이터가 없을 경우에 '03'으로 뜨는데, 이 경우 no_data_flag = 1로 지정
    if jdata['response']['header']['resultCode'] == '03':
        jdata = ''
        p_args['no_data_flag'] = 1    
        return p_args
    
    else:
        p_args['no_data_flag'] = 0
        json_filename = today + '_' + nx + '_' + ny + '.json'
        with open(os.path.join(tempPath, json_filename), 'w') as f:
            json.dump(jdata, f)
            p_args['json_filename'] = json_filename 
        
        return p_args


# JSON 파일을 데이터프레임으로 변환하고 CSV 파일로 저장 
def save_result(p_args):
    
    today = p_args['today']
    tempPath = p_args['tempPath']
    json_filename = p_args['json_filename']
        
    with open(os.path.join(tempPath, json_filename), 'r') as f:
        jdata = json.load(f)
    # print(jdata)
    # print(json.dumps(jdata, indent=2)[:400])
    
    jdata = jdata['response']['body']['items']['item']

    all_result = pd.DataFrame()
    for row in jdata:
        row_list = []
        row_list.append(row)
        each_df = pd.DataFrame(row_list)
        all_result = pd.concat([all_result, each_df], axis=0)
    all_result = all_result.reset_index(drop=True)
    # print(all_result.head(2))
    
    all_result.to_parquet(os.path.join(tempPath , 'result_' + json_filename.replace('.json', '') + '.gzip'),
                      compression='gzip', index=None)
    return p_args


# 관측소 결과 모두 취합해서 Parquet 파일로 저장
def merge_all_result(p_args):
   
    today = dict_args['today']
    tempPath = p_args['tempPath']
    resultPath = p_args['resultPath']

    all_df = pd.DataFrame()
    csvfile_list = [f for f in glob.glob(os.path.join(tempPath, 'result_*.gzip'))]
    for file in csvfile_list:
        each_df = pd.read_parquet(file)
        # print(each_df.head(1))
        all_df = pd.concat([all_df, each_df], axis=0)
    all_df = all_df.reset_index(drop=True)
    all_df = all_df.fillna("")
    
    # Parquet 파일로 저장
    all_df.to_parquet(os.path.join(resultPath, 'result_all_regions' + '_' + today + '.gzip'), 
                                   compression='gzip', index=None)
    



if __name__ == '__main__':
    
    # <-------------------1번째 Step-------------------> #
    area_master = pd.read_csv(dict_args['area_master'], sep=',')
    area_master = area_master.head(3)
    
    count = 1
    
    for idx, row in area_master.iterrows():
        
        start_time = time.time()
        
        print('-'*120)
        time.sleep(2)
        nx = row['nx']
        ny = row['ny']
        print('X, Y 좌표: ', nx, ',', ny)
        
        # 기상청 단기예보 조회서비스 (최근 3일 데이터만 가능)
        # X,Y 좌표별 데이터 수집
        # *일일 트래픽 10000건까지
        params = {'serviceKey' : 'BL2wBSzTux87JT8DVgN59Hsiz/po3FS/yt4YE+HT6zoxiJFKMuN3oEZ0XKMOhSp64HiDZSO/gsI17IfFUPtX1g==', 
                  'numOfRows' : '',
                  'pageNo' : '1',  
                  'dataType' : 'json', 
                  'base_date' : '20220630', 
                  'base_time' : '0800',
                  'nx' : nx, 
                  'ny' : ny,
                  }
        
        dict_args['nx'] = str(nx) 
        dict_args['ny'] = str(ny)
             
        # 10개 관측소 데이터 수집하면 잠깐 쉬어주기
        if count % 10 == 0:
            time.sleep(4)

        # API 접속이 끊길시 계속 반복해서 접속 시도    
        time.sleep(4)    
        try:
            dict_args = get_data_api(dict_args, **params)
            print('데이터 수집 완료')
        except:
            time.sleep(4)
            try:
                dict_args = get_data_api(dict_args, **params)
                print('데이터 수집 완료')
            except:
                time.sleep(4)
                try:
                    dict_args = get_data_api(dict_args, **params)
                    print('데이터 수집 완료')
                except:
                    pass            
                
        count += 1
        print('시간 소요: ', time.time() - start_time, ' seconds')
        
        # <-------------------2번째 Step-------------------> #
        no_data_flag = dict_args['no_data_flag']
        if no_data_flag == 1:
            print('--------------데이터가 없습니다--------------')
            continue
        else:
            time.sleep(4)
            dict_args = save_result(dict_args)
   
        # <-------------------3번째 Step-------------------> #
        merge_all_result(dict_args)
        

------------------------------------------------------------------------------------------------------------------------
X, Y 좌표:  60 , 127
연결되었습니다.  <Response [200]>
http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey=BL2wBSzTux87JT8DVgN59Hsiz%2Fpo3FS%2Fyt4YE%2BHT6zoxiJFKMuN3oEZ0XKMOhSp64HiDZSO%2FgsI17IfFUPtX1g%3D%3D&numOfRows=&pageNo=1&dataType=json&base_date=20220630&base_time=0800&nx=60&ny=127
데이터 수집 완료
시간 소요:  6.207953453063965  seconds
------------------------------------------------------------------------------------------------------------------------
X, Y 좌표:  61 , 127
연결되었습니다.  <Response [200]>
http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey=BL2wBSzTux87JT8DVgN59Hsiz%2Fpo3FS%2Fyt4YE%2BHT6zoxiJFKMuN3oEZ0XKMOhSp64HiDZSO%2FgsI17IfFUPtX1g%3D%3D&numOfRows=&pageNo=1&dataType=json&base_date=20220630&base_time=0800&nx=61&ny=127
데이터 수집 완료
시간 소요:  6.17634391784668  seconds
-------------------------------------------------

gzip 파일이름은 그때 그때 마다 생성되는게 달라짐

In [11]:
total=pd.read_parquet("C:\\kim\\weather\\result_all_regions_220630_1717.gzip")

In [13]:
total

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20220630,0800,TMP,20220630,0900,25,60,126
1,20220630,0800,UUU,20220630,0900,0.4,60,126
2,20220630,0800,VVV,20220630,0900,2.6,60,126
3,20220630,0800,VEC,20220630,0900,189,60,126
4,20220630,0800,WSD,20220630,0900,2.6,60,126
5,20220630,0800,SKY,20220630,0900,4,60,126
6,20220630,0800,PTY,20220630,0900,1,60,126
7,20220630,0800,POP,20220630,0900,60,60,126
8,20220630,0800,WAV,20220630,0900,0,60,126
9,20220630,0800,PCP,20220630,0900,5.0mm,60,126


In [12]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   baseDate   30 non-null     object
 1   baseTime   30 non-null     object
 2   category   30 non-null     object
 3   fcstDate   30 non-null     object
 4   fcstTime   30 non-null     object
 5   fcstValue  30 non-null     object
 6   nx         30 non-null     int64 
 7   ny         30 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 2.0+ KB


In [14]:
# < ----------------- area_master 테이블 축소 ----------------- >
# def cleansing_area_master(p_args):
#     file_path = dict_args['resultPath']

#     raw_area_master = pd.read_csv(dict_args['area_master'], sep='\t')
#     raw_area_master = raw_area_master.astype(str)
#     raw_area_master['XY'] = raw_area_master['격자 X'] + '_' + raw_area_master['격자 Y'] 
#     raw_area_master = raw_area_master['XY'].unique().tolist()
#     # 1628개의 X, Y 좌표
#     # print(area_master['XY'].nunique())

#     area_master = pd.DataFrame()
#     for row in raw_area_master:
#         nx = row.split('_')[0]
#         ny = row.split('_')[1]
#         each_df = pd.DataFrame({'nx': [nx], 'ny': [ny]})
#         area_master = pd.concat([area_master, each_df], axis=0)
#     area_master = area_master.reset_index(drop=True)
#     area_master.to_csv(os.path.join(file_path, 'area_master_small.txt'), index=None)